In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers.merge import concatenate

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow import keras

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD
import cv2, numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_datagen_with_aug = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=[0.2,0.5],
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   validation_split=0.2)

train_generator = train_datagen_with_aug.flow_from_directory(
    r'/content/drive/My Drive/X-Ray Images Dataset for research project-20211204T084221Z-001',
    target_size=(331, 331),
    batch_size=32,
    # color_mode='grayscale',
    class_mode='categorical',
    subset='training'
    ) # set as training data

validation_generator = train_datagen_with_aug.flow_from_directory(
    r'/content/drive/My Drive/X-Ray Images Dataset for research project-20211204T084221Z-001', # same directory as training data
    target_size=(331, 331),
    batch_size=32,
    # color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
    ) # set as validation data

input_img = Input(shape=(331, 331, 3))

Found 5171 images belonging to 3 classes.
Found 1292 images belonging to 3 classes.


In [ ]:
from keras.applications import nasnet

NASNET_LARGE = Sequential()

nasnetlarge_model =keras.applications.nasnet.NASNetLarge(
    input_shape=(331,331,3), include_top=False, weights='imagenet',
    pooling=('max'), classes=3
)

343621632/343610240 [==============================] - 2s 0us/step


In [ ]:
for layer in nasnetlarge_model.layers:
    layer.trainable = False
    
NASNET_LARGE.add(nasnetlarge_model)
NASNET_LARGE.add(Flatten())
NASNET_LARGE.add(Dense(512,activation='relu'))
NASNET_LARGE.add(Dense(3, activation='softmax'))

NASNET_LARGE.summary()

Adam = keras.optimizers.Adam(learning_rate=0.001)
NASNET_LARGE.compile(optimizer=Adam, loss='categorical_crossentropy', metrics=['accuracy'])

graph = NASNET_LARGE.fit(train_generator, epochs=25, validation_data = validation_generator)

graph

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 4032)              84916818  
                                                                 
 flatten (Flatten)           (None, 4032)              0         
                                                                 
 dense (Dense)               (None, 512)               2064896   
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 86,983,253
Trainable params: 2,066,435
Non-trainable params: 84,916,818
_________________________________________________________________
Epoch 1/25


In [ ]:
import matplotlib.pyplot as plt

fig1 = plt.gcf()

plt.plot(graph.history['accuracy'])
plt.plot(graph.history['val_accuracy'])

# plt.axis(ymin=0.4, ymax=1)
plt.grid()

plt.title('NASNet Large Accuracy Input: 224x224 for COVID-19 Dataset')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train','validation'])

plt.show()

In [ ]:
max_acc = max(graph.history['val_accuracy'])

print('The highest accuracy achieved using NASNet Large Model with input 224x224 is', max_acc*100)